In [ ]:
import stata_setup 
import pandas as pd
stata_setup.config("/Applications/STATA","se")

In [ ]:
%%stata

pwd

In [ ]:
%%stata

cd "~/Documents/Pitt/Year 3/TA - Econ 3080/econ-3080-recitations/Recitation 0 - Intro to Stata"


In [ ]:
%%stata 

import delimited "https://github.com/brunokomel/econ-3080-recitations/blob/main/Recitation%200%20-%20Intro%20to%20Stata/metrics_fall_2023_grades.csv", clear


• $\textbf{Browse}$ ($\texttt{browse [varlist]}$): Opens data browser to view these variables

In [ ]:
%%stata 

browse

In [ ]:
%%stata -doutd df1 

import delimited "https://raw.githubusercontent.com/brunokomel/econ-3080-recitations/main/Recitation%200%20-%20Intro%20to%20Stata/metrics_fall_2023_grades.csv", clear


In [ ]:
display(df1)

In [ ]:
%%stata -doutd df1 

import excel "https://github.com/brunokomel/econ-3080-recitations/raw/main/Recitation%200%20-%20Intro%20to%20Stata/metrics_fall_2023_grades.xlsx", clear

In [ ]:
display(df1)

In [ ]:
%%stata

sysdir

# Data Cleaning

In [ ]:
%%stata -doutd df1

sysuse auto, clear

In [ ]:
display(df1)

In [ ]:
%%stata 

summarize price mpg headroom if foreign == 0 & rep78 == 1 

sum price mpg headroom if foreign == 0 & rep78 == 2

In [ ]:
%%stata

bysort rep78: sum price mpg headroom if foreign == 0

$\textbf{Order}$ $(\texttt{order [varlist]}$): Relocates columns in data browser $\\$
$\textbf{Sort}$ $(\texttt{sort [varlist]}$): Sorts row by values (e.g. highest to lowest
wages)

In [ ]:
%%stata -doutd df1

sysuse auto

In [ ]:
%%stata -doutd df2

order make mpg price

sort mpg

In [ ]:
display(df1)


In [ ]:
display(df2)

$\textbf{Tag duplicates}$ ($\texttt{duplicates tag [varlist]}$): Generates variable representing number of duplicates for particular value (e.g. ID numbers)

In [ ]:
%%stata -doutd df2 

duplicates tag mpg , gen(dup_mpg)

In [ ]:
display(df2)

In [ ]:
%%stata

gen price_per_pound = price / weight
replace price_per_pound = 1 if mpg > 15 | foreign != 1

In [ ]:
%%stata

sum weight

return list

In [ ]:
%%stata -doutd df2 

quietly sum weight  // quietly (or, equivalently, qui) suppresses the output

local avg_weight = r(mean)

gen heavy = 0
replace heavy = 1 if weight >= `avg_weight'

In [ ]:
display(df2)

In [ ]:
%%stata

sum heavy

sum dup_mpg

In [ ]:
%%stata  -doutd df2

label define wgt_cat 1 "Heavy" 0 "Light"
label values heavy wgt_cat

In [ ]:
display(df2)

# Data Inspection

In [ ]:
%%stata

sum price mpg

In [ ]:
%%stata

tab foreign

In [ ]:
%%stata

tab foreign heavy

In [ ]:
%%stata 

pwcorr foreign heavy


# Hypothesis Testing

In [ ]:
%%stata

ttest weight, by(foreign)

return list
power twomeans `r(mu_1)' `r(mu_2)', sd(`r(sd_1)')

In [ ]:
%%stata

ttest mpg == weight

# Regression

In [ ]:
%%stata

reg price weight rep78 foreign

In [ ]:
%%stata

vif  // testing Multicolinearity (Some sources say VIF >=5 is too high, others say that 10 is an appropriate cut-off)

In [ ]:
%%stata

reg price weight rep78#foreign //partial interaction (only includes interaction terms)


reg price weight rep78##foreign  // full interaction, includes interaction as well as individual terms

In [ ]:
%%stata

estat hettest //low p-value suggests presence of heteroskedasticity

In [ ]:
%%stata -doutd df2

gen sample_reg = e(sample) //constructs dummy variable equal to 1 if obs included in regression

In [ ]:
display(df2)

# Loops 

In [ ]:
%%stata
foreach v of varlist weight rep78 foreign{
    reg price `v'
}

In [ ]:
%%stata 

forvalues i = 1/5{
    reg price weight foreign if rep78 == `i'
}

In [ ]:
%%stata

local i = 1 

while `i' <= 5{
    reg price weight foreign if rep78 == `i'
    local i = `i' + 1
}

In [ ]:
%%stata

levelsof rep78, local(levels)

foreach i of local levels{
    reg price weight foreign if rep78 == `i'
}

## Postestimation Results Retrieval (Important)

In [ ]:
%%stata

reg price weight foreign

In [ ]:
%%stata

ereturn list

In [ ]:
%%stata

matrix list e(b)

In [ ]:
%%stata

disp e(b)[1,1]

# Other Stuff

In [ ]:
%%stata

preserve

In [ ]:
%%stata -doutd df3

collapse (mean) price weight foreign (median) m_price = price, by(rep78)

In [ ]:
display(df3)

In [ ]:
%%stata -doutd df3 

restore

In [ ]:
display(df3)

# Common Errors

In [ ]:
%%stata

unique make

In [ ]:
%%stata

gen make_new = make + 2

// Operations with variables of different types/classes

In [ ]:
%%stata

gen newvar = . 

reg price newvar weight

// No valid observations in a regression

# What about plots?

In [ ]:
display(df2)

In [ ]:
%%stata 

set scheme s2color

In [ ]:
%%stata

graph twoway scatter weight length
graph save wl, replace

In [ ]:
%%stata

set scheme gg_tableau

graph twoway scatter weight length
graph save wl, replace

In [ ]:
%%stata

graph twoway scatter price mpg
graph save pm, replace

In [ ]:
%%stata

graph combine wl.gph pm.gph

In [ ]:
%%stata

set scheme white_tableau

In [ ]:
%%stata

graph twoway (scatter price mpg) (lfit price mpg)

In [ ]:
%%stata

hist price, freq

In [ ]:
%%stata

graph box price


In [ ]:
%%stata

graph bar price, by(foreign)